In [1]:
import seaborn as sns
import pandas as pd

In [2]:
!pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.2 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 2.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Dataset Column Information

| Field Name | Description                                                        | Data Type    |
|------------|--------------------------------------------------------------------|--------------|
| age        | Age of the patient                                                 | Integer      |
| sex        | Gender of the patient (1 = male; 0 = female)                       | Categorical  |
| cp         | Chest pain type (e.g., typical angina, atypical angina, etc.)      | Categorical  |
| trestbps   | Resting blood pressure (in mm Hg)                                  | Integer      |
| chol       | Serum cholesterol in mg/dl                                         | Integer      |
| fbs        | Fasting blood sugar > 120 mg/dl (1 = true; 0 = false)              | Categorical  |
| restecg    | Resting electrocardiographic results                               | Categorical  |
| thalach    | Maximum heart rate achieved                                        | Integer      |
| exang      | Exercise-induced angina (1 = yes; 0 = no)                          | Categorical  |
| oldpeak    | ST depression induced by exercise relative to rest                 | Integer      |
| slope      | Slope of the peak exercise ST segment                              | Categorical  |
| ca         | Number of major vessels (0-3) colored by fluoroscopy               | Integer      |
| thal       | Thalassemia (e.g., normal, fixed defect, reversible defect)        | Categorical  |
| num        | Diagnosis of heart disease (0 = no disease; 1+ = presence of HD)   | Integer      |


The machine learning solution for this dataset aims to predict the presence of heart disease using key patient attributes like age, cholesterol levels, and exercise-induced responses. This model can assist healthcare professionals in making early and accurate diagnoses, reducing the need for costly diagnostic procedures, and enabling targeted patient care. By identifying high-risk individuals, the solution supports efficient resource allocation and improves patient outcomes.

In [3]:
pip install sqlalchemy


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 6.7 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 5.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from sqlalchemy import create_engine, text

In [12]:
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'MSDS610' # db we just created
user = r'postgres' # using the postgres user for this demo
pw = r'1234' # this is the password established during installation
port = r'5433' # default port estabalished during install
schema = r'raw' # schema we just created

In [13]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [18]:
# Define table schema and create the table in the raw schema
create_table_query = """
CREATE SCHEMA IF NOT EXISTS raw;

CREATE TABLE IF NOT EXISTS raw.HeartDisease (
    age INTEGER,
    sex VARCHAR(10),
    cp VARCHAR(50),
    trestbps INTEGER,
    chol INTEGER,
    fbs BOOLEAN,
    restecg VARCHAR(50),
    thalach INTEGER,
    exang BOOLEAN,
    oldpeak NUMERIC,
    slope VARCHAR(50),
    ca INTEGER,
    thal VARCHAR(50),
    num INTEGER
);
"""

# Execute the SQL query
with db_conn.connect() as connection:
    connection.execute(text(create_table_query))
    print("Table created successfully!")


Table created successfully!


# Loading csv file to database

In [26]:
# importing csv file
csv_file_path = "final_dataset.csv"  

df = pd.read_csv(csv_file_path)
df.to_sql(
    name="HeartDisease",      
    con=db_conn,                  
    schema="raw",                
    if_exists="append",          
    index=False                 
)

print("CSV data imported successfully!")


CSV data imported successfully!


# Retriving Data from table

In [25]:
schema = "raw"  
table_name = "HeartDisease"  

sql = f'SELECT * FROM "{schema}"."{table_name}"'

data_check = pd.read_sql(sql, db_conn, index_col=None)
print(data_check.head())  

   age  sex  cp trestbps  chol fbs restecg thalach exang oldpeak slope ca   
0   32    1   1       95     0   ?       0     127     0      .7     1  ?  \
1   34    1   4      115     0   ?       ?     154     0      .2     1  ?   
2   35    1   4        ?     0   ?       0     130     1       ?     ?  ?   
3   36    1   4      110     0   ?       0     125     1       1     2  ?   
4   38    0   4      105     0   ?       0     166     0     2.8     1  ?   

  thal  num  
0    ?    1  
1    ?    1  
2    7    3  
3    6    1  
4    ?    2  
